In [6]:
import requests
import pandas as pd              
import numpy as np               
import matplotlib.pyplot as plt  
import time
from tqdm import tqdm_notebook
import json


token = 'ac77745bac77745bac77745b97ac1c3072aac77ac77745bf1797d3f23a6cef1fd575e58'
myid = '181118707'
version = '5.82'

# Файл с группами ВК
filepath = 'groups.txt'

# Количество обрабатываемых постов (с начала ленты) за раз (максимум - 100)
num_of_posts_to_show = 100
    

def get_groups_ids(filepath):
    """Считываем ID групп Вконтакта из текстового файла"""
    try:
        with open(filepath) as groups_data:
            groups = groups_data.read().split('\n') # считываем информацию из файла
            groups = list(filter(None, groups)) # удаление пустых строк
            return groups
    except FileNotFoundError:
        print('File with groups is not found!')
        return None


def get_posts(group, response):
    """Получаем датафрейм с информацией по изображениям и текстам в постах"""
    
    posts = []
    dates = []
    photos = []
    texts = []
    for post in range(num_of_posts_to_show):
        posts.append('vk.com/club' + str(group))
        texts.append(response['response']['items'][post]['text'])
        post = response['response']['items'][post]
        dates.append(post['date'])
        try:
            photos.append(post['attachments'][0]['photo']['sizes'][-1]['url'])
        except:
            photos.append('NO_PHOTOS')
            continue
    
    d = {'photos': photos, 'dates': dates, 'posts': posts, 'texts': texts}
    df = pd.DataFrame(data=d)
    return df


def vk_download(method, parameters, token=token):
    url = 'https://api.vk.com/method/'+method+'?'+parameters+'&access_token='+str(token)+'&v='+version
    response = requests.get(url) 
    return(response.json())


    
def get_group_info(group):
    """Получаем информацию по текстам и изображениям в постах"""
    
    r = requests.get("https://api.vk.com/method/wall.get", params={"owner_id": '-{}'.format(group), 
                                                                   "count": num_of_posts_to_show,
                                                                   "access_token": token, "v": version})
    response = r.json()
    posts = get_posts(group, response)
    return posts


def get_posts_num(group):
    """Получить общее число постов в группе"""
    
    n = vk_download('wall.get','owner_id=-{}'.format(group))['response']['count']
    return n


def parse_vk():
    groups = get_groups_ids(filepath)
    df = pd.DataFrame()
    for group in groups:
        n = get_posts_num(group) # суммарное количество постов в группе
        print(n)
        for i in tqdm_notebook(range(0, n+100, 100)): # ВК может за раз сохранить информацию лишь о ста постах
            # После каждого запроса будем ждать 0.4 секунды, иначе вк не отдаст данные. 
            time.sleep(0.4)
            # Получаем датафрейм для одной группы
            group_data = get_group_info(group)
            # Сцепляем данные из всех групп
            df = pd.concat([df, group_data], ignore_index=True)
    print(df)
    df.to_csv('/Users/user/Desktop/aylifind/df.csv', index=False, encoding='utf-8')


parse_vk()

18141



588



                                                  photos       dates  \
0                                              NO_PHOTOS  1561749033   
1                                              NO_PHOTOS  1561740571   
2                                              NO_PHOTOS  1561740565   
3                                              NO_PHOTOS  1561740559   
4                                              NO_PHOTOS  1561735765   
5                                              NO_PHOTOS  1561731687   
6      https://sun1-15.userapi.com/c855420/v855420125...  1561729500   
7      https://sun1-17.userapi.com/c858036/v858036361...  1561717374   
8      https://sun1-86.userapi.com/c850620/v850620459...  1561708260   
9      https://sun1-17.userapi.com/c854320/v854320548...  1561705206   
10                                             NO_PHOTOS  1561702451   
11                                             NO_PHOTOS  1561699166   
12     https://sun1-15.userapi.com/c855136/v855136241...  15616

Сделать обработку для групп меньше чем с ```num_of_posts_to_show``` количеством записей

In [4]:
a = pd.read_csv('/Users/user/Desktop/aylifind/df.csv')
a

,photos,dates,posts,texts
0,NO_PHOTOS,1561749033,vk.com/club67878782,Здравствуйте. Где в Заокском можно наточить но...
1,NO_PHOTOS,1561740571,vk.com/club67878782,"Здравствуйте! Подскажите,пожалуйста, в Заокско..."
2,NO_PHOTOS,1561740565,vk.com/club67878782,"Здравствуйте!\nПорекомендуйте, пожалуйста, инс..."
3,NO_PHOTOS,1561740559,vk.com/club67878782,Добрый день. У беспроводной колонки сломался р...
4,NO_PHOTOS,1561735765,vk.com/club67878782,Здравствуйте. Подскажите хорошего и надёжного ...
5,NO_PHOTOS,1561731687,vk.com/club67878782,"Внимание, внимание! Изменения!!!"
6,https://sun1-15.userapi.com/c855420/v855420125...,1561729500,vk.com/club67878782,"Подслушано на Турслете💪💪💪 да, да, и до туда до..."
7,https://sun1-17.userapi.com/c858036/v858036361...,1561717374,vk.com/club67878782,Дети нашли кошелёк на улице Мира 4. Вернём хоз...
8,https://sun1-86.userapi.com/c850620/v850620459...,1561708260,vk.com/club67878782,Уже завтра)) не пропустите 😉
9,https://sun1-17.userapi.com/c854320/v854320548...,1561705206,vk.com/club67878782,NaN


# Как парсить  

In [3]:
def vk_download(method, parameters, token=token):
    url = 'https://api.vk.com/method/'+method+'?'+parameters+'&access_token='+str(token)+'&v='+version
    response = requests.get(url) 
    return(response.json())


# Проверяем всё ли работает на своём имени.
vk_download('users.get', 'user_ids='+myid)

{'response': [{'id': 181118707,
   'first_name': 'Andrey',
   'last_name': 'Aslanov'}]}